In [1]:
!pip install requests beautifulsoup4 --upgrade --quiet
!pip install mysql-connector-python --upgrade --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import time
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import mysql.connector

urls = [
    'https://www.amazon.in/Henlo-Supplement-Vitamin-Nutrition-Support/dp/B09STSYFTP/ref=sr_1_1?keywords=henlo+topper&qid=1686568008&sr=8-1',
    'https://www.amazon.in/Henlo-Human-Grade-Ingredients-Nutritionally-Balanced/dp/B0BVRVLJJ9/ref=sr_1_1_sspa?crid=BFDCEXXPBCZ1&keywords=henlo%2Bdog%2Bfood&qid=1686553729&s=pet-supplies&sprefix=henlo%2Bdog%2Bfoo%2Cpets%2C333&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1',
    'https://www.amazon.in/Ykibble-Dog-Food-Adult-Nutritionally/dp/B09WF68DVF?th=1',
    'https://www.amazon.in/Wiggles-EveryDawg-Adult-Chicken-Vegetables/dp/B0BQ6SH6HJ/ref=zg_bs_17169008031_sccl_1/257-2965700-7927855?psc=1'
       ] 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}

mydb = mysql.connector.connect(host="35.200.229.177", user="analyticsuser", password="PI-fZOdeMu*0fiG|", database="analytics")
# Create a cursor object
mycursor = mydb.cursor()

In [3]:
def send_email(subject, message):
    # Email configuration
    sender_email = 'rahul.alovespets@supertails.com'  # Your Gmail email address
    receiver_email = 'rahul.alovespets@supertails.com'  # Recipient email address

    # SMTP server configuration
    smtp_server = 'smtp.gmail.com'  # Gmail SMTP server
    smtp_port = 587  # SMTP server port number
    smtp_username = 'rahul.alovespets@supertails.com'  # Your Gmail email address
    smtp_password = 'Rahul@3784'  # Your Gmail account password

    # Create a MIME message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the email message
    msg.attach(MIMEText(message, 'plain'))

    try:
        # Create a SMTP session
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Enable TLS encryption
            server.login(smtp_username, smtp_password)  # Login to the SMTP server
            server.sendmail(sender_email, receiver_email, msg.as_string())  # Send the email
        print('Email sent successfully!')
    except Exception as e:
        print('Error sending email:', str(e))

In [4]:
def scrape():
          
        empty_lst=[]
        for index, url in enumerate(urls):
            resp = requests.get(url, headers=headers)
            time.sleep(5)

            if resp.status_code != 200:
                send_email('URL Error', f"The URL at index {index} returned a status code of {resp.status_code}. URL: {url}")
                continue

            soup = BeautifulSoup(resp.text)
            time.sleep(5)
            print(resp.status_code)

            time_stramp = datetime.datetime.now()
            title = soup.find('span', {'id': 'productTitle'}).text.strip()
            table = soup.find('table', {'id': 'productDetails_detailBullets_sections1'})
            trs = table.find_all('tr')
            rank = trs[-1].find('td')
            tds = rank.find_all('span')[0].text.strip()
            print(tds)
            print(title)
            print(time_stramp)
            # Store the product info in a dictionary
            product_dict = { "timestamp": time_stramp, "product_title": title, "best_seller_rank": tds }
            empty_lst.append(product_dict)
            time.sleep(3)
        return empty_lst

data=scrape()

200
#314 in Pet Supplies (See Top 100 in Pet Supplies)  #10 in Dog Supplements & Vitamins  #35 in Home
Henlo Everyday Topper | Joint Support, Heart, Skin & Coat and Gut Health | Dog Supplement | Human Grade Ingredients | All Life Stages | 100 gm Pack
2023-06-15 10:29:35.663418
200
#282 in Pet Supplies (See Top 100 in Pet Supplies)  #34 in Home  #47 in Dry Dog Food
Henlo Baked Dry Food for Adult Dogs | 100% Human-Grade Ingredients | Nutritionally Balanced Everyday Dog Food | Made in India | Chicken Flavour | 1KG
2023-06-15 10:29:51.440949
200
#1,792 in Pet Supplies (See Top 100 in Pet Supplies)  #143 in Dry Dog Food
WIGGLES Ykibble Dog Food Adult Pet, 900 gm - Oven Baked Nutritionally Balanced Food - Chicken, Vegetables, Vitamins & Minerals
2023-06-15 10:30:07.424985
200
#333 in Pet Supplies (See Top 100 in Pet Supplies)  #50 in Dry Dog Food
Wiggles EveryDawg Dry Adult Dog Food, 1.2kg (Chicken, Oats & Vegetables)
2023-06-15 10:30:23.248220


In [5]:
def insert_data_to_sql(data):
    # Convert list of dictionaries to list of tuples
    tuples_data = [(d['timestamp'], d['product_title'], d['best_seller_rank']) for d in data]
    val=[]
    for tuple_item in tuples_data:
        val.append(tuple_item)

    tuple(val)

    # Prepare the SQL statement
    sql = "INSERT INTO henloAmazonRank (timestamp, products, bestSellerRank) VALUES (%s, %s, %s)"

    # Execute the SQL statement
    mycursor.executemany(sql, tuples_data)

    # Commit the changes
    mydb.commit()

    # Print the number of records inserted
    print(mycursor.rowcount, "record(s) inserted.")

In [6]:
insert_data_to_sql(data)

4 record(s) inserted.
